In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import beta
import plotly.express as px
from datetime import datetime
from dotenv import load_dotenv
from snowflake import connector
import warnings
warnings.filterwarnings('ignore', category=Warning)

In [2]:
brands = pd.read_csv('/Users/peter/Downloads/shopper_ltv.csv')
brands.sample(5)

,BRAND
3,Caraway
11,D.S. & Durga
35,Canopy
12,Licorice
26,Sol de Janeiro


In [6]:
brand = 'Caden Lane'
brand_cohort_query = f""" 
with 
disco_emails as (
    select 
        email,
        to_date(min(order_date)) as order_date
    from disco_mixpanel.master_conversions
    where 
        order_date between '2023-01-01' and '2022-01-31'
        and conversion_brand = '{brand}'
    group by all),
order_emails as (
    select 
        odf.email, 
        to_date(min(created_at_gmt)) as first_order_date
    from orders.order_data_flattened odf 
    left join postgres.core_publisher cp
        on odf.publisher = cp.remote_id
    left join postgres.core_brand cb 
        on cb.core_brand_id = cp.brand_id
    left join disco_emails de 
        on de.email = odf.email 
    where 
        de.email is not null 
        and cb.name = '{brand}'
    group by all
        ),
brand_cohort as (
    select 
        distinct oe.email
    from order_emails oe 
    inner join disco_emails de 
        on oe.email = de.email and oe.first_order_date = de.order_date
    order by random(42)
    limit 50
    ),
conv_emails as (  
    select 
        distinct odf.email 
    from derived.conversion c 
    left join orders.order_data_flattened odf 
        on c.remote_order_id = odf.order_id
    union all 
    select 
        distinct email
    from disco_mixpanel.master_conversions
    ),
    --- 2. create a random sample of non-Disco emails with purchases in Q3 2022
random_cohort as (    
    select 
        distinct email
    from orders.order_data_flattened odf 
    left join conv_emails ce using (email)
    where 
        ce.email is null 
        and to_date(created_at_gmt) between '2023-01-01' and '2022-01-31'
    order by random(42)
    limit 50
),
cohort_emails as (
    select 
        email,
        'disco' as cohort
    from brand_cohort
    union all 
    select 
        email,
        'random' as cohort
    from random_cohort
    )
    select 
        year(created_at_gmt) as year,
        month(created_at_gmt) as month, 
        cohort,
        count(distinct email) as purchase_users,
        count(order_id) as orders,
        sum(order_cost) as total_order_cost
    from orders.order_data_flattened odf 
    inner join cohort_emails using (email)
    where 
        to_date(created_at_gmt) >= '2023-02-01'
    group by all 
    order by year, month, cohort asc;
    """

In [5]:
# from dotenv import load_dotenv
load_dotenv('/Users/peter/.env')


# establish Snowflake connection
connection = connector.connect(user=os.getenv("SNOWFLAKE_USERNAME"), 
                  password=os.getenv("SNOWFLAKE_PASSWORD"), 
                  account=os.getenv("SNOWFLAKE_ACCOUNT"), 
                  role=os.getenv("SNOWFLAKE_ROLE"), 
                  warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
                  database = 'DISCO_CORE' 
                  )

# execute the query
# shopper_data = pd.read_sql(brand_cohort_query, connection)
# shopper_data.head()
caden_lane_ltv = pd.read_sql(brand_cohort_query, connection)

DatabaseError: Execution failed on sql ' 
with 
disco_emails as (
    select 
        email,
        to_date(min(order_date)) as order_date
    from disco_mixpanel.master_conversions
    where 
        order_date between '2023-01-01' and '2022-01-31'
        and conversion_brand = 'Caden Lane'
    group by all),
order_emails as (
    select 
        odf.email, 
        to_date(min(created_at_gmt)) as first_order_date
    from orders.order_data_flattened odf 
    left join postgres.core_publisher cp
        on odf.publisher = cp.remote_id
    left join postgres.core_brand cb 
        on cb.core_brand_id = cp.brand_id
    left join disco_emails de 
        on de.email = odf.email 
    where 
        de.email is not null 
        and cb.name = 'Caden Lane'
    group by all
        ),
brand_cohort as (
    select 
        distinct oe.email
    from order_emails oe 
    inner join disco_emails de 
        on oe.email = de.email and oe.first_order_date = de.order_date
    order by random(42)

In [19]:
def run_ltv(df):
    output = []
    for brand in df.BRAND:
        #Run the brand cohort query for each brand
        # Get the cohort count first
        brand_cohort_query_count = f"""
         with disco_emails AS (SELECT 
                email,
                to_date(min(order_date)) as order_date
            FROM disco_mixpanel.master_conversions
            WHERE 
                order_date BETWEEN '2022-05-01' AND '2022-05-31'
                AND conversion_brand = '{brand}'
            GROUP BY email
        ),
        order_emails AS (SELECT 
                odf.email, 
                to_date(min(created_at_gmt)) as first_order_date
            FROM orders.order_data_flattened odf 
            LEFT JOIN postgres.core_publisher cp ON odf.publisher = cp.remote_id
            LEFT JOIN postgres.core_brand cb ON cb.core_brand_id = cp.brand_id
            LEFT JOIN disco_emails de ON de.email = odf.email 
            WHERE 
                de.email IS NOT NULL 
                AND cb.name = '{brand}'
            GROUP BY odf.email
        ),
        brand_cohort AS (SELECT 
                distinct oe.email
            FROM order_emails oe 
            INNER JOIN disco_emails de ON oe.email = de.email AND oe.first_order_date = de.order_date
            LIMIT 50
        )SELECT COUNT(*) FROM brand_cohort;
        """
        brand_cohort_count = pd.read_sql(brand_cohort_query_count, connection).iloc[0, 0]
        
        brand_cohort_query = f"""
        with 
        disco_emails as (
            select 
                email,
                to_date(min(order_date)) as order_date
            from disco_mixpanel.master_conversions
            where 
                order_date between '2022-05-01' and '2022-05-31'
                and conversion_brand = '{brand}'
            group by all),
        order_emails as (
            select 
                odf.email, 
                to_date(min(created_at_gmt)) as first_order_date
            from orders.order_data_flattened odf 
            left join postgres.core_publisher cp
                on odf.publisher = cp.remote_id
            left join postgres.core_brand cb 
                on cb.core_brand_id = cp.brand_id
            left join disco_emails de 
                on de.email = odf.email 
            where 
                de.email is not null 
                and cb.name = '{brand}'
            group by all
                ),
        brand_cohort as (
            select 
                distinct oe.email
            from order_emails oe 
            inner join disco_emails de 
                on oe.email = de.email and oe.first_order_date = de.order_date
            order by random(42)
            limit 50
            ),
        conv_emails as (  
            select 
                distinct odf.email 
            from derived.conversion c 
            left join orders.order_data_flattened odf 
                on c.remote_order_id = odf.order_id
            union all 
            select 
                distinct email
            from disco_mixpanel.master_conversions
            ),
            --- 2. create a random sample of non-Disco emails with purchases in Q3 2022
        random_cohort as (    
            select 
                distinct email
            from orders.order_data_flattened odf 
            left join postgres.core_publisher cp
                on odf.publisher = cp.remote_id
            left join postgres.core_brand cb 
                on cb.core_brand_id = cp.brand_id
            left join conv_emails ce using (email)
            where 
                ce.email is null 
                and to_date(created_at_gmt) between '2022-05-01' and '2022-05-31'
                and cb.name = '{brand}'
            order by random(42)
            limit {brand_cohort_count}
        ),
        cohort_emails as (
            select 
                email,
                'disco' as cohort
            from brand_cohort
            union all 
            select 
                email,
                'random' as cohort
            from random_cohort
            ),
        cohort_sizes as (
            select
                cohort,
                count(distinct email) as cohort_size
            from cohort_emails
            group by cohort
          )
            select 
                year(created_at_gmt) as year,
                month(created_at_gmt) as month, 
                cohort,
                cohort_size,
                count(distinct email) as purchase_users,
                count(order_id) as orders,
                sum(order_cost) as total_order_cost
            from orders.order_data_flattened odf 
            inner join cohort_emails using (email)
            left join cohort_sizes using (cohort)
            where 
                to_date(created_at_gmt) >= '2022-06-01'
            group by all 
            order by year, month, cohort asc;
        """
        # execute the query
        shopper_data = pd.read_sql(brand_cohort_query, connection)
        #Create a column called brand in the shopper_data dataframe
        shopper_data['brand'] = brand
        #move the brand column to the front of the dataframe
        cols = shopper_data.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        shopper_data = shopper_data[cols]
        output.append(shopper_data)
        #Print a progress message for every 25% of brands processed
        if len(output) % int(len(df.BRAND)/4) == 0:
            print(f'{len(output)} brands processed')
    
    #Concatenate the output list of dataframes into a single dataframe
    final_df = pd.concat(output)
    return final_df

df = run_ltv(brands)

10 brands processed
20 brands processed
30 brands processed
40 brands processed


In [21]:
df.to_csv('/Users/peter/Downloads/ltv_output.csv', index=False)
df.sample(5)

,brand,YEAR,MONTH,COHORT,COHORT_SIZE,PURCHASE_USERS,ORDERS,TOTAL_ORDER_COST
20,Cadence,2023,4,disco,50,13,28,4536.61
4,Needed,2022,8,disco,37,16,30,1980.39
42,Canopy,2024,3,disco,10,3,7,547.06
34,Amberjack,2023,11,disco,50,11,20,2451.33
25,Hedley & Bennett,2023,7,disco,15,1,1,374.49


In [22]:
df

,brand,YEAR,MONTH,COHORT,COHORT_SIZE,PURCHASE_USERS,ORDERS,TOTAL_ORDER_COST
0,Lalo,2022,6,disco,31,15,51,5544.45
1,Lalo,2022,6,random,31,7,9,1186.33
2,Lalo,2022,7,disco,31,13,24,2802.28
3,Lalo,2022,7,random,31,5,6,490.68
4,Lalo,2022,8,disco,31,14,32,2717.25
...,...,...,...,...,...,...,...,...
7,Curie,2022,12,random,2,1,1,37.56
8,Curie,2023,3,random,2,1,1,55.71
9,Curie,2023,7,disco,2,1,1,46.00
10,Curie,2023,10,random,2,1,1,118.81
